In [1]:
from PIL import Image, ImageDraw
import torch
import torchvision.transforms.functional as T
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader
from object_detection_data import Encoder
from pytorch_lightning.callbacks import RichProgressBar
import pytorch_lightning as pl
from object_detection_models import MultiClassJetNet


In [2]:
class SyntheticData(DataLoader):
    def __init__(self, image_width, image_height, length, encoder):
        self.image_width = image_width
        self.image_height = image_height
        self.length = length
        self.encoder = encoder

        self.images = []
        self.encoded_bounding_boxes = []
        self.encoded_target_classes = []
        self.target_masks = []
        for _ in range(length):
            image, bounding_box = self._generate_image()
            encoded_bounding_boxes, target_mask, target_classes = self.encoder.apply(
                bounding_box, torch.tensor([[1]])
            )
            self.images.append(image)
            self.encoded_bounding_boxes.append(encoded_bounding_boxes)
            self.encoded_target_classes.append(target_classes)
            self.target_masks.append(target_mask)

    def __getitem__(self, idx):
        return (
            self.images[idx],
            self.encoded_bounding_boxes[idx],
            self.target_masks[idx],
            self.encoded_target_classes[idx],
            idx,
        )

    def __len__(self) -> int:
        return self.length

    def _generate_image(self):
        image = Image.new("1", (image_width, image_height))
        image_draw = ImageDraw.Draw(image)
        center = torch.tensor(
            [
                torch.randint(0, image_width - 1, (1,)).item(),
                torch.randint(0, image_height - 1, (1,)).item(),
            ]
        )
        size = torch.tensor([image_width * 0.25, image_height * 0.25])
        upper_left = center - size / 2
        lower_right = center + size / 2
        image_draw.rectangle(
            [upper_left[0], upper_left[1], lower_right[0], lower_right[1]], fill=255
        )
        bounding_box = torch.tensor(
            [
                [
                    center[0].item() / image_width,
                    center[1].item() / image_height,
                    size[0].item() / image_width,
                    size[1].item() / image_height,
                ]
            ]
        )
        return T.pil_to_tensor(image).to(torch.float), bounding_box


image_width = 80
image_height = 60
torch.manual_seed(2)

default_scalings = torch.tensor([[0.25, 0.25]])
num_classes = 1
encoder = Encoder(default_scalings, num_classes)

data = SyntheticData(image_width, image_height, 10000, encoder)
data_loader = DataLoader(data, batch_size=64, shuffle=False, drop_last=True)


In [3]:
from pytorch_lightning import loggers as pl_loggers

tb_logger = pl_loggers.TensorBoardLogger(save_dir="synthetic_data_logs/")


pl_model = MultiClassJetNet(encoder, 1e-3)
trainer = pl.Trainer(
    limit_predict_batches=100, max_epochs=60, callbacks=[RichProgressBar()], logger = tb_logger
)
trainer.fit(model=pl_model, train_dataloaders=data_loader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/jonathan/hulks/ml/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(


┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃    ┃ Name                   ┃ Type               ┃ Params ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0  │ accuracy               │ MulticlassAccuracy │      0 │
│ 1  │ batch_normalization_1  │ BatchNorm2d        │      2 │
│ 2  │ conv2d_1               │ Conv2d             │    160 │
│ 3  │ batch_normalization_2  │ BatchNorm2d        │     32 │
│ 4  │ depthwise_conv2d_1     │ Conv2d             │    144 │
│ 5  │ conv2d_2               │ Conv2d             │    408 │
│ 6  │ batch_normalization_3  │ BatchNorm2d        │     48 │
│ 7  │ depthwise_conv2d_2     │ Conv2d             │    216 │
│ 8  │ conv2d_3               │ Conv2d             │    400 │
│ 9  │ batch_normalization_4  │ BatchNorm2d        │     32 │
│ 10 │ depthwise_conv2d_3     │ Conv2d             │    144 │
│ 11 │ conv2d_4               │ Conv2d             │    340 │
│ 12 │ batch_normalization_5  │ BatchNorm2d        │     40 │
│ 13 │ depthwise_conv2d_4     │ Conv2d             │    180 │
│ 14 │ conv2d_5               │ Conv2d             │    420 │
│ 15 │ batch_normalization_6  │ BatchNorm2d        │     40 │
│ 16 │ depthwise_conv2d_5     │ Conv2d             │    180 │
│ 17 │ conv2d_6               │ Conv2d             │    420 │
│ 18 │ batch_normalization_7  │ BatchNorm2d        │     40 │
│ 19 │ depthwise_conv2d_6     │ Conv2d             │    180 │
│ 20 │ conv2d_7               │ Conv2d             │    420 │
│ 21 │ batch_normalization_8  │ BatchNorm2d        │     40 │
│ 22 │ depthwise_conv2d_7     │ Conv2d             │    180 │
│ 23 │ conv2d_8               │ Conv2d             │    420 │
│ 24 │ batch_normalization_9  │ BatchNorm2d        │     40 │
│ 25 │ depthwise_conv2d_8     │ Conv2d             │    180 │
│ 26 │ conv2d_9               │ Conv2d             │    504 │
│ 27 │ batch_normalization_10 │ BatchNorm2d        │     48 │
│ 28 │ conv2d_10              │ Conv2d             │  5.2 K │
│ 29 │ batch_normalization_11 │ BatchNorm2d        │     48 │
│ 30 │ conv2d_11              │ Conv2d             │  5.2 K │
│ 31 │ batch_normalization_12 │ BatchNorm2d        │     48 │
│ 32 │ conv2d_12              │ Conv2d             │  5.2 K │
│ 33 │ batch_normalization_13 │ BatchNorm2d        │     48 │
│ 34 │ conv2d_13              │ Conv2d             │  5.2 K │
│ 35 │ conv2d_14              │ Conv2d             │    150 │
└────┴────────────────────────┴────────────────────┴────────┘

Trainable params: 26.4 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 26.4 K                                                                                               
Total estimated model params size (MB): 0

2023-03-02 11:14:28.417134: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 11:14:28.591100: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-02 11:14:29.331164: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-02 11:14:29.331231: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

Output()

`Trainer.fit` stopped: `max_epochs=20` reached.
